In [6]:
import transformers
from datasets import load_dataset
import numpy as np
import torch

from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, Pipeline
import tensorflow as tf

## **tensorflow 训练方法**

In [6]:
dataset_dir = r'./hf/dataset/glue'
dataset = load_dataset(dataset_dir, 'cola')
# dataset = load_dataset('glue', 'cola')

Found cached dataset glue (/root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 802.58it/s]


<function evaluate.loading.load(path: str, config_name: Union[str, NoneType] = None, module_type: Union[str, NoneType] = None, process_id: int = 0, num_process: int = 1, cache_dir: Union[str, NoneType] = None, experiment_id: Union[str, NoneType] = None, keep_in_memory: bool = False, download_config: Union[datasets.download.download_config.DownloadConfig, NoneType] = None, download_mode: Union[datasets.download.download_manager.DownloadMode, NoneType] = None, revision: Union[str, datasets.utils.version.Version, NoneType] = None, **init_kwargs) -> evaluate.module.EvaluationModule>

In [5]:
examples = dataset['train'].with_format(type='pandas')
examples[:5]

,sentence,label,idx
0,"Our friends won't buy this analysis, let alone...",1,0
1,One more pseudo generalization and I'm giving up.,1,1
2,One more pseudo generalization or I'm giving up.,1,2
3,"The more we study verbs, the crazier they get.",1,3
4,Day by day the facts are getting murkier.,1,4


In [4]:
metric = load('glue')
# metric

FileNotFoundError: Couldn't find a module script at /mnt/workspace/glue/glue.py. Module 'glue' doesn't exist on the Hugging Face Hub either.

In [8]:
#模型下载
from modelscope.hub.snapshot_download import snapshot_download
model_dir = snapshot_download('AI-ModelScope/distilbert-base-uncased')
print(model_dir)

/mnt/workspace/.cache/modelscope/AI-ModelScope/distilbert-base-uncased


In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_dir,)

In [10]:
def preprocess_func(example):
    token = tokenizer(example['sentence'],truncation=True)
    return token

In [22]:
dataset_tokenized = dataset.map(preprocess_func, batched=True)

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e77f116dc26d05b1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e0dda4e16a316c5f.arrow


In [23]:
dataset_tokenized['test'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['unacceptable', 'acceptable'], id=None),
 'idx': Value(dtype='int32', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [11]:
id2label = {0: "Grammar_Wrong", 1: "Grammar_Right"}
label2id = {value: key for key, value in id2label.items()}

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_dir, num_labels=2, id2label=id2label, label2id=label2id
)

2023-12-06 15:43:46.219822: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-06 15:43:46.268879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-06 15:43:46.269139: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
tf_train_dataset = model.prepare_tf_dataset(
    dataset_tokenized["train"],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    dataset_tokenized["validation"],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [32]:
from transformers import create_optimizer
batch_size = 16
num_epochs = 20
batches_per_epoch = len(dataset_tokenized["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer, metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir="./text_classification_model_save/logs")

checkpoint_filepath = './text_classification_model_save/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
callbacks = [tensorboard_callback,model_checkpoint_callback]

In [39]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=callbacks,
)

Epoch 1/20
534/534 [==============================] - 19s 35ms/step - loss: 0.0057 - accuracy: 0.9975 - val_loss: 1.3801 - val_accuracy: 0.8063
Epoch 2/20
534/534 [==============================] - 17s 32ms/step - loss: 0.0050 - accuracy: 0.9979 - val_loss: 1.3801 - val_accuracy: 0.8063
Epoch 3/20
534/534 [==============================] - 17s 32ms/step - loss: 0.0047 - accuracy: 0.9974 - val_loss: 1.3801 - val_accuracy: 0.8063
Epoch 4/20
534/534 [==============================] - 17s 32ms/step - loss: 0.0041 - accuracy: 0.9981 - val_loss: 1.3801 - val_accuracy: 0.8063
Epoch 5/20
534/534 [==============================] - 17s 32ms/step - loss: 0.0042 - accuracy: 0.9980 - val_loss: 1.3801 - val_accuracy: 0.8063
Epoch 6/20
534/534 [==============================] - 17s 32ms/step - loss: 0.0042 - accuracy: 0.9979 - val_loss: 1.3801 - val_accuracy: 0.8063
Epoch 7/20
534/534 [==============================] - 17s 32ms/step - loss: 0.0054 - accuracy: 0.9975 - val_loss: 1.3801 - val_accuracy:

In [14]:
# 调出训练过的checkpoint:
model.load_weights(checkpoint_filepath)

In [48]:
test_sentence=["do u have good one?",
              "it is the last words",
              "The judge told that the jurors to think carefully",
              "I is a good ",
              "I ask him to do job",]
output = model(tokenizer(test_sentence,return_tensors='tf',padding='longest'))

In [49]:
output

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[-2.3238587 ,  2.843852  ],
       [-3.6311479 ,  4.199532  ],
       [ 4.212416  , -3.8671856 ],
       [-0.16245975,  0.487365  ],
       [ 4.2147355 , -3.8299022 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [51]:
classfications = tf.argmax(output.logits, axis=1)

classifications = [model.config.id2label[c.numpy()] for c in classfications]
print(classifications)

['Grammar_Right', 'Grammar_Right', 'Grammar_Wrong', 'Grammar_Right', 'Grammar_Wrong']


## **pytorch training:**

In [3]:
# 下载model, datasets
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torchmetrics
from modelscope import snapshot_download

model_dir = snapshot_download('AI-ModelScope/bert-base-cased')
print(f"model下载目录:{model_dir}")
model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=5) # yelp_review_full dataset的类别有5个;

from datasets import load_dataset
dataset = load_dataset("./hf/dataset/yelp_review_full")
print(dataset["train"][100])

2023-12-17 18:34:29,667 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-12-17 18:34:29,670 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2023-12-17 18:34:29,671 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-12-17 18:34:29,835 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 a8a545858e67690dd394730b698bbb8b and a total number of 945 components indexed


model下载目录:/mnt/workspace/.cache/modelscope/AI-ModelScope/bert-base-cased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /mnt/workspace/.cache/modelscope/AI-ModelScope/bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Found cached dataset yelp_review_full (/root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:02<00:00,  1.42s/it]

{'label': 0, 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. I

In [42]:
# dataset process:

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True) 

def tokenize_function(examples):
    """
    对dataset每个样本,进行truncation;padding;
    truncation必须这里完成并且设定max_length;
    padding可以在collate_fn中,再padding,实现batch内部padding,而不是整个datasets中padding;
    """
    # return_tensors=None : 返回 list
    # return_tensors='py' : 返回 tensor
    return tokenizer(examples["text"], padding=False, truncation=True, return_tensors=None,max_length=512) # bert-base-cased模型,参照model card的config.json:  "max_position_embeddings": 512, 

def tokenize_function_batch(examples):
    """
    DataLoader中的collate_fn: 函数中实现: 
    padding/truncation/dataloder
    """
    texts = [tokenizer(ex["text"], padding=True, truncation=True, max_length=512) for ex in examples]
    labels = [ex['label'] for ex in examples]            
    return {'label':labels,
            'text': texts}

tokenized_dataset = dataset.map(tokenize_function, batched=False, batch_size=1000, num_proc=8, load_from_cache_file=True) # cpu的逻辑核心可以通过os.cpu_count()获取; batch_size缺省值即为1000,可以省略;
train_dataset = tokenized_dataset['train'].shuffle(seed=11).select(range(5000))
test_dataset = tokenized_dataset['train'].shuffle(seed=11).select(range(1000))

Loading cached processed dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-1b394405ec9298c1_*_of_00008.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-64fa14a8037aa111_*_of_00008.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-6a9eb5bb43e8054b.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-6a9eb5bb43e8054b.arrow


In [33]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [34]:
data_sample_iter = iter(tokenized_dataset['train'])

In [35]:
data_sample = next(data_sample_iter)
for item in data_sample.items():
    if item[0] != 'label':
        print(f"len:{len(item[1])} ; {item} \n")
    else:
        print(f"len:{item[1]} ; {item} \n")

len:4 ; ('label', 4) 

len:534 ; ('text', "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.") 

len:124 ; ('input_ids', [101, 173, 1197, 119, 2284, 2953, 3272, 1917, 178, 1440, 1111, 1107, 170, 1704, 22351, 119, 1119, 112, 188, 3505, 1105, 3123, 1106, 2037, 1106, 1443, 1217, 10063, 4404, 132, 1119, 112, 188, 1579, 1113, 1159, 1107, 3195, 1117, 4420, 132, 1119, 112, 188, 6559, 1114, 170, 1499, 118, 23555, 2704, 113, 183, 9379, 114, 1134, 1139, 2153, 1138, 3716, 1106, 1143, 1110, 1304, 1696, 1107, 1692, 1380, 5

In [36]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
output = model(**encoded_input)
print(output)

{'input_ids': tensor([[  101, 20777, 17510,  1143,  1118,  1251,  3087,  1128,   112,   173,
          1176,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.6522, -0.0350,  0.2353,  0.6221, -0.7457]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


### <font color=lime size=8  face=kaiti>**pytorch dataset process 心得:**</font>
> + torch.utilis.data.DataLoader是torch原生的重要的将Dataset进行Load进入train的重要方法. 其中的collate_fn是每个dataset样本,或者batch的list用于转换的函数,其接收一个batch的list,然后生成一个带有label的字典,
> + 这个函数转换的结果再送入DataLoader生成model.forward()所需要的input_ids,其为一个字典,但仅仅保留三个keys:input_ids,token_type_ids,attention_mask,其它key(或者是column)已经被移除;
> + transformers.DataCollatorWithPadding,其实就是一个具体的(instanciate)DataLoader()中的collate_fn,只不过,其仅仅完成padding的功能,不对dataset进行任何embedding/tokenizer的动作;所以:
> + 所以,在transformers的训练时的wrap后的Trainer类而言, 实现dataset的process可以两个方法: 
### <font color=lime size=5  face=kaiti>标准的方法是: </font>
> 1. 首先datasets.map(tokenizer, num_proc=n)对每个dataset进行tokenizer,此时,必须要truncation=True,max_length=max_length), 
> 2. 然后transformers.Trainer(data_collator=transformers.DataCollatorWithPadding, dataset=tokenized_dataset);
> 3. 缺点是map过程计算会事先将所有的数据都tokenizer一次,非常耗时,虽然有num_proc可以利用CPU的多进程,是不能使用GPU的;
### <font color=lime size=5  face=kaiti>或者方法二:</font>
> 1. transfomers.Trainer()中的dataset可以送入原始的dataset对象,并不占用多少内存,不事先进行tokenizer;而是将包含tokenizer, padding等等过程全部放入data_collator中;
> 2. transformers.Trainer()中的data_collator其实相当于torch.utilis.data.DataLoader原生类中的collate_fn,
> 3. 所以,需要自己写一个函数来实现单样本数据或者batch的dataset送入collate_fn,进行tokenizer,padding,truncation等一系列操作,再生成一个字典
> 4. 这样, transformers.Trainer()中的dataset, 首先进行collate_fn,进行process,然后,data_collator就会将dataset转换成process后的生成器,并且符合model.forward()的input的格式要求

In [26]:
from torch.utils.data import DataLoader
data = DataLoader(dataset['test'],batch_size=8,shuffle=False, collate_fn=tokenize_function_batch, prefetch_factor=2, num_workers=8)

In [55]:
next(iter(data))['text'][-1]

{'input_ids': [101, 1573, 119, 6335, 119, 16068, 106, 165, 183, 165, 183, 2240, 160, 6258, 3048, 146, 1180, 1505, 170, 1461, 1120, 1103, 3968, 1104, 170, 6131, 119, 6466, 2213, 146, 1309, 1261, 3267, 8497, 1105, 146, 112, 173, 1930, 1322, 1146, 23993, 6647, 1544, 1103, 4017, 1168, 1190, 1103, 8220, 119, 12998, 1122, 112, 188, 8881, 183, 112, 153, 8043, 1161, 112, 188, 156, 5114, 4184, 119, 165, 183, 165, 183, 1942, 27516, 1110, 170, 1541, 1632, 1282, 1106, 1301, 1165, 1128, 1138, 1149, 1104, 1411, 6065, 1137, 1132, 1702, 1111, 170, 4106, 1282, 1106, 1321, 1800, 1111, 170, 5913, 7978, 1137, 170, 4106, 1480, 1149, 119, 1135, 112, 188, 1198, 1380, 1472, 1106, 1202, 1168, 1190, 171, 5658, 6185, 1122, 1146, 1120, 1103, 6668, 1113, 170, 4306, 1480, 119, 165, 183, 165, 183, 2428, 17059, 1107, 1713, 1128, 112, 1325, 1129, 6573, 1213, 170, 109, 126, 118, 1275, 2267, 4882, 3451, 8898, 1105, 2094, 1128, 1243, 119, 1109, 1363, 1645, 1110, 1103, 2094, 1110, 1909, 1121, 1103, 4309, 2977, 19984, 2341

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /mnt/workspace/.cache/modelscope/AI-ModelScope/bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
# metrics 函数:
def compute_metrics(p: transformers.EvalPrediction) -> dict:
    """
    torchmetrics类实现类别的precision,recall,f1_score, accuracy:
    1. transformers.Trainer()中,生成的transformer.EvalPrediction的输出为ndarray类型,转换为tensor为torchmetrics库使用;
    2. 转换成字典;
    也可以使用sklearn.metrics.precision_recall_fscore_support函数实现,该函数要求input为ndarray.
    """
    preds,labels=p
    # preds = np.argmax(preds, axis=-1)
    
    preds = torch.from_numpy(preds)
    labels= torch.from_numpy(labels)
    
    precision = torchmetrics.functional.precision(preds,labels,task='multiclass',num_classes=5)
    recall = torchmetrics.functional.recall(preds,labels,task='multiclass',num_classes=5)
    f1 = torchmetrics.functional.f1_score(preds,labels,task='multiclass',num_classes=5)
    accuracy = torchmetrics.functional.accuracy(preds,labels,task='multiclass',num_classes=5)
    
    # precision, recall, f1, _ = precision_recall_fscore_support(labels.flatten(), preds.flatten(), average='weighted', zero_division=0)
   
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [43]:
from transformers import TrainingArguments, Trainer

batch_size=8
EarlyStoppingCallBack = transformers.EarlyStoppingCallback(early_stopping_patience=30,)

training_args = TrainingArguments(
    output_dir='test_trainer',
    evaluation_strategy = "steps",
    eval_steps = 50,
    save_strategy = "steps",
    save_steps =500,
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    group_by_length=True,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    dataloader_num_workers=8,
    # push_to_hub=True,
)

collate_fn = transformers.DataCollatorWithPadding(tokenizer=tokenizer,padding=True,max_length=None,return_tensors='pt') # 当tokenizer参数有输入时,data_collator缺省为DataCollatorWithPadding,因而该句可以省略
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn, # 当tokenizer参数有输入时,data_collator缺省为DataCollatorWithPadding,因而该句可以省略
    train_dataset=train_dataset,    
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallBack],
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `_

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,No log,0.962969,0.622000,0.622000,0.622000,0.622000
100,No log,1.123250,0.514000,0.514000,0.514000,0.514000
150,No log,1.033388,0.576000,0.576000,0.576000,0.576000
200,No log,0.936984,0.598000,0.598000,0.598000,0.598000
250,No log,0.915111,0.599000,0.599000,0.599000,0.599000
300,No log,0.929415,0.602000,0.602000,0.602000,0.602000
350,No log,0.835709,0.643000,0.643000,0.643000,0.643000
400,No log,0.809572,0.668000,0.668000,0.668000,0.668000
450,No log,0.755530,0.696000,0.696000,0.696000,0.696000
500,0.997200,0.748524,0.689000,0.689000,0.689000,0.689000


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `_

In [26]:
checkpoint = 'test_trainer/checkpoint-3000'
FT_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True) 
# FT_pipeline = Pipeline(task='text-classification',model=FT_model, tokenizer=tokenizer,)

我们可以将Yelp Review Full数据集中的评分类别解释为以下五个类别：

1. 评分为1：非常差评（Very negative）
2. 评分为2：差评（Negative）
3. 评分为3：中性评价（Neutral）
4. 评分为4：好评（Positive）
5. 评分为5：非常好评（Very positive）

In [38]:
text = "what a beautiful girl !!!!!, I can't love you more!!!!!"
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
FT_model.eval() # 设置为评估模式,去除dropout
with torch.no_grad():
    output = FT_model(**encoded_input)
print(output)
torch.argmax(output.logits)

{'input_ids': tensor([[ 101, 1184,  170, 2712, 1873,  106,  106,  106,  106,  106,  117,  146,
         1169,  112,  189, 1567, 1128, 1167,  106,  106,  106,  106,  106,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[-1.3139, -2.1106, -2.5306, -1.1107,  5.0567]]), hidden_states=None, attentions=None)


tensor(4)